# Config & Imports

In [1]:
config = {
    'data': {
        'n_datasets': 1, # the number of datasets
        
        'n_samples': 10_000, # the number of samples per dataset
        
        'n_features': 10, # the number of features
        
        'n_informative': 10,
        # The number of informative features, i.e., the number of features used to build the linear model used to generate the output.
        
        'n_targets': 2,
        # The number of regression targets, i.e., the dimension of the y output vector associated with a sample. By default, the output is a scalar.
    
        'bias': 0.0,
        # The bias term in the underlying linear model
        
        'effective_rank': 5,
        # if not None:
            # The approximate number of singular vectors required to explain most of the input data by linear combinations. Using this kind of singular spectrum in the input allows the generator to reproduce the correlations often observed in practice.
        # if None:
            # The input set is well conditioned, centered and gaussian with unit variance.
        
        'tail_strength': 0.5,
        # The relative importance of the fat noisy tail of the singular values profile if effective_rank is not None. When a float, it should be between 0 and 1.
        
        'noise': 0.0,
        # The standard deviation of the gaussian noise applied to the output.
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': None,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.2,
                'val_size': 0.1,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 100,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 10,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 1,   
    }
}

In [2]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization

import utilities

# Load Data

In [3]:
X_datasets_list = np.zeros([config['data']['n_datasets'], config['data']['n_samples'], config['data']['n_features']])

if  config['data']['n_targets'] < 2:
    y_datasets_list = np.zeros([config['data']['n_datasets'], config['data']['n_samples'], ])
    coef_list = np.zeros([config['data']['n_datasets'], config['data']['n_features'], ])
else:
    y_datasets_list = np.zeros([config['data']['n_datasets'], config['data']['n_samples'], config['data']['n_targets']])
    coef_list = np.zeros([config['data']['n_datasets'], config['data']['n_features'], config['data']['n_targets']])

In [4]:
directory = utilities.data_path(config)

with open(directory + '/X_datasets_list.npy', "rb") as f:
    X_datasets_list = np.load(f, allow_pickle=True)
with open(directory + '/y_datasets_list.npy', "rb") as f:
    y_datasets_list = np.load(f, allow_pickle=True)
with open(directory + '/coef_list.npy', "rb") as f:
    coef_list = np.load(f, allow_pickle=True)

In [5]:
# X_datasets_list[0]

# Prepare Data (Functions)

In [6]:
def train_test_val_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size=config['lambda']['data_prep']['train_test_val_split']['test_size'] + config['lambda']['data_prep']['train_test_val_split']['val_size'], 
                                                        random_state=config['lambda']['data_prep']['train_test_val_split']['random_state'], 
                                                        shuffle=config['lambda']['data_prep']['train_test_val_split']['shuffle'], 
                                                        stratify=config['lambda']['data_prep']['train_test_val_split']['stratify'])
    X_test, X_val, y__test, y_val = train_test_split(X_test, 
                                                    y_test, 
                                                    test_size=config['lambda']['data_prep']['train_test_val_split']['val_size'] / (config['lambda']['data_prep']['train_test_val_split']['test_size'] + config['lambda']['data_prep']['train_test_val_split']['val_size']), 
                                                    random_state=config['lambda']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config['lambda']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config['lambda']['data_prep']['train_test_val_split']['stratify'])
    return X_train, X_test, X_val, y_train, y_test, y_val
    

# Save Model & Metrics (functions)

In [7]:
def save_model(model, datasetIndex):
    directory = utilities.lambda_path(config, datasetIndex)
    

    # with open(directory + '.npy', "wb") as f:
    model.save(directory)

# Train Model

In [8]:
# for i in range(10):
for i in range(config['data']['n_datasets']):
    # Data Prep
    X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X_datasets_list[i],
                                                                          y_datasets_list[i])
    
    # Model Def
    model = Sequential()
    model.add(BatchNormalization(input_dim=config['data']['n_features']))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(config['data']['n_targets'], activation='sigmoid'))

    
    model.compile(optimizer='adam', loss='mae', metrics=['mae', keras.metrics.RootMeanSquaredError()])
    model.summary()
    
    # Model fit
    history = model.fit(x=X_train,
                        y=y_train,
                        batch_size=config['lambda']['model_fit']['batch_size'],
                        epochs=config['lambda']['model_fit']['epochs'],
                        verbose=config['lambda']['model_fit']['verbose'],
                        callbacks=config['lambda']['model_fit']['callbacks'],
                        validation_data=(X_val, y_val),
                        shuffle=config['lambda']['model_fit']['shuffle'],
                        class_weight=config['lambda']['model_fit']['class_weight'],
                        sample_weight=config['lambda']['model_fit']['sample_weight'],
                        initial_epoch=config['lambda']['model_fit']['initial_epoch'],
                        steps_per_epoch=config['lambda']['model_fit']['steps_per_epoch'],
                        validation_steps=config['lambda']['model_fit']['validation_steps'],
                        validation_batch_size=config['lambda']['model_fit']['validation_batch_size'],
                        validation_freq=config['lambda']['model_fit']['validation_freq'],
                       )
    print(history.history['val_loss'])
    save_model(model, i)

2022-05-04 16:53:41.180385: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-04 16:53:49.369030: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46723 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6
2022-05-04 16:53:49.370484: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 8735 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:25:00.0, compute capability: 8.6
2022-05-04 16:53:49.372009: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 wi

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 10)               40        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 100)               1100      
                                                                 
 dense_1 (Dense)             (None, 60)                6060      
                                                                 
 dense_2 (Dense)             (None, 2)                 122       
                                                                 
Total params: 7,322
Trainable params: 7,302
Non-trainable params: 20
_________________________________________________________________
Epoch 1/100
 24/219 [==>...........................] - ETA: 0s - loss: 0.9392 - mae: 0.9392 - root_mean_squared_error: 

2022-05-04 16:53:52.559923: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


219/219 [==============================] - 4s 8ms/step - loss: 0.7168 - mae: 0.7168 - root_mean_squared_error: 0.9555 - val_loss: 0.7990 - val_mae: 0.7990 - val_root_mean_squared_error: 1.0241
Epoch 2/100
219/219 [==============================] - 1s 5ms/step - loss: 0.5990 - mae: 0.5990 - root_mean_squared_error: 0.8531 - val_loss: 0.7141 - val_mae: 0.7141 - val_root_mean_squared_error: 0.9245
Epoch 3/100
219/219 [==============================] - 1s 5ms/step - loss: 0.5984 - mae: 0.5984 - root_mean_squared_error: 0.8522 - val_loss: 0.6067 - val_mae: 0.6067 - val_root_mean_squared_error: 0.8475
Epoch 4/100
219/219 [==============================] - 1s 5ms/step - loss: 0.5919 - mae: 0.5919 - root_mean_squared_error: 0.8500 - val_loss: 0.5336 - val_mae: 0.5336 - val_root_mean_squared_error: 0.8291
Epoch 5/100
219/219 [==============================] - 1s 5ms/step - loss: 0.5935 - mae: 0.5935 - root_mean_squared_error: 0.8503 - val_loss: 0.5414 - val_mae: 0.5414 - val_root_mean_squared_e

2022-05-04 16:55:33.031472: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: data/nda1_nsa10000_nfe10_nin10_nta2_bia0.0_eff5_tai0.5_noi0.0_shuTrue_ranNone/0_tsi0.2_vsi0.1_ranNone_shuTrue_strNone_bat32_epo100_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1/assets


# Inspect Metrics